In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
import os
import random
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error


import xgboost as xgb
from surprise import Reader, Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import GridSearchCV

In [ ]:
if not os.path.isfile("/content/drive/MyDrive/NetflixProject/Netflix/archive"): 
#This line: "os.path.isfile("../Data/NetflixRatings.csv")" simply checks that is there a file with the name "NetflixRatings.csv" in the 
#in the folder "/Data/". If the file is present then it return true else false
    startTime = datetime.now()
    data = open("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixRatings.csv", mode = "w") #this line simply creates the file with the name "NetflixRatings.csv" in 
    #write mode in the folder "Data".
#     files = ['../Data/combined_data_1.txt','../Data/combined_data_2.txt', '../Data/combined_data_3.txt', '../Data/combined_data_4.txt']
    files = ['/content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_1.txt','/content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_2.txt',
             '/content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_3.txt', '/content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_4.txt']
    for file in files:
        print("Reading from file: "+str(file)+"...")
        with open(file) as f:  #you can think of this command "with open(file) as f" as similar to 'if' statement or a sort of 
            #loop statement. This command says that as long as this file is opened, perform the underneath operation.
            for line in f:
                line = line.strip() #line.strip() clears all the leading and trailing spaces from the string, as here each line
                #that we are reading from a file is a string.
                #Note first line consist of a movie id followed by a semi-colon, then second line contains custID,rating,date
                #then third line agains contains custID,rating,date which belong to that movie ID and so on. The format of data
                #is exactly same as shown above with the heading "Example Data Point". Check out above.
                if line.endswith(":"):
                    movieID = line.replace(":", "") #this will remove the trailing semi-colon and return us the leading movie ID.
                else:
                    #here, in the below code we have first created an empty list with the name "row "so that we can insert movie ID 
                    #at the first position and rest customerID, rating and date in second position. After that we have separated all 
                    #four namely movieID, custID, rating and date with comma and converted a single string by joining them with comma.
                    #then finally written them to our output ".csv" file.
                    row = [] 
                    row = [x for x in line.split(",")] #custID, rating and date are separated by comma
                    row.insert(0, movieID)
                    data.write(",".join(row))
                    data.write("\n")
        print("Reading of file: "+str(file)+" is completed\n")
    data.close()
    print("Total time taken for execution of this code = "+str(datetime.now() - startTime))

Reading from file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_1.txt...
Reading of file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_1.txt is completed

Reading from file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_2.txt...
Reading of file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_2.txt is completed

Reading from file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_3.txt...
Reading of file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_3.txt is completed

Reading from file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_4.txt...
Reading of file: /content/drive/MyDrive/NetflixProject/Netflix/archive/combined_data_4.txt is completed

Total time taken for execution of this code = 0:03:03.509629


In [ ]:
# creating data frame from our output csv file.
if not os.path.isfile("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixData.pkl"):
    startTime = datetime.now()
    Final_Data = pd.read_csv("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixRatings.csv", sep=",", names = ["MovieID","CustID", "Ratings", "Date"])
    Final_Data["Date"] = pd.to_datetime(Final_Data["Date"])
    Final_Data.sort_values(by = "Date", inplace = True)
    print("Time taken for execution of above code = "+str(datetime.now() - startTime))

Time taken for execution of above code = 0:01:38.323984


In [ ]:
# storing pandas dataframe as a picklefile for later use
if not os.path.isfile("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixData.pkl"):
    Final_Data.to_pickle("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixData.pkl")
else:
    Final_Data = pd.read_pickle("/content/drive/MyDrive/NetflixProject/Netflix/archive/NetflixData.pkl")

In [ ]:
Final_Data.shape

(100480507, 4)

In [ ]:
Final_Data.head()

,MovieID,CustID,Ratings,Date
56431994,10341,510180,4,1999-11-11
9056171,1798,510180,5,1999-11-11
58698779,10774,510180,3,1999-11-11
48101611,8651,510180,2,1999-11-11
81893208,14660,510180,2,1999-11-11
